# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

In [25]:
import pandas as pd
import numpy as np

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.** 

In [26]:
#ваш код 
df = pd.read_csv('data/Advertising2.csv', sep=',')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head(5)

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [27]:
#ваш код 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


In [28]:
#ваш код 
df.isnull().mean()

TV           0.0
radio        0.0
newspaper    0.0
sales        0.0
dtype: float64

**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y(целевая переменная)** 

In [29]:
#ваш код 
y = df['sales'].values
X = df.drop('sales', axis=1)
X = X.values

print(f'Type of X: {type(X)}')
print(f'Type of y: {type(y)}')

Type of X: <class 'numpy.ndarray'>
Type of y: <class 'numpy.ndarray'>


## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [30]:
import numpy as np

X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
y = y.reshape(-1, 1)
print(X.shape, y.shape)

(200, 4) (200, 1)


**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [31]:
X = X / np.sqrt(np.sum(np.square(X), axis=0))

**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм координатного спуска для случая нормализованных данных:

**Задано:**

* $X=(x_{ij})$ - матрица наблюдений, размерностью $dim(X)=(n, m)$
* $N=1000$ - количество итераций

**Примечание:** *1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.*

**Алгоритм (математическая запись):**
* Создать нулевой вектор параметров $w_0=(0, 0,..., 0)^T$
* Для всех $t=1, 2, ..., N$ итераций:
    * Для всех $k = 1, 2,..., m$:
        * Фиксируем значение всех признаков, кроме $k$-ого и вычисляем прогноз модели линейной регрессии.Для этого исключаем признак $k$-ый из данных и $w_j$ из параметров при построении прогноза.
        Математически это можно записать следующим образом:

        $$h_i = \sum_{j=1}^{k-1} x_{ij}w_{j} + \sum_{j=k+1}^{m} x_{ij}w_j $$

        **Примечание:**
        
        *Обратите, что в данной записи текущий признак под номером $k$ не участвует в сумме.Сравните эту запись с классической записью прогноза линейной регрессии в случае нормированных данных (когда участвуют все признаки):*

        $$h_i = \sum_{j=1}^{m} x_{ij}w_{j}$$ 
        
        * Вычисляем новое значение параметра $k$-ого коэффициента: 
        $$w_k = \sum_{i=1}^{n} x_{ik} (y_i - h_i) = x_k^T(y-h) $$

    * Вычисляем значение функции потерь и сохраняем в историю изменения функции потерь (В оценке функции потерь участвуют все признаки):
        $$\hat{y_i} = \sum_{j=1}^{m}x_{ij}$$
        $$Loss_t = \frac{1}{n} \sum_{i=1}^{n}(y_i-\hat{y_i})^2$$
        
        или в векторном виде:
        
        $$\hat{y} = Xw$$
        $$Loss_t = \frac{1}{n}(y-\hat{y})^T(y-\hat{y})$$
    



**Алгоритм (псевдокод):**
```python

num_iters = #количество итераций
m = # количество строк в матрице X
n = # количество столбцов в матрице X
w = #вектор размера nx1, состояющий из нулей

for i in range(num_iters):
    for k in range(n):
        # Вычисляем прогноз без k-ого фактора
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        # Обновляем новое значение k-ого коэффициента
        w[k] =  (X[:,k].T @ (y - h))
        # Вычисляем функцию потерь
        cost = sum((X @ w) - y) ** 2)/(len(y))

```

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [32]:
# ваш код 
num_iters = 1000 #количество итераций
m = X.shape[0] # количество строк в матрице X
n = X.shape[1] # количество столбцов в матрице X
w = np.zeros(X.shape[1]) #вектор размера nx1, состояющий из нулей

for i in range(num_iters):
    for k in range(n):
        # Вычисляем прогноз без k-ого фактора
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        # Обновляем новое значение k-ого коэффициента
        w[k] =  (X[:,k].T @ (y - h.reshape(-1, 1)))
        # Вычисляем функцию потерь
        cost = sum(((X @ w) - y) ** 2)/(len(y))

display(w)

array([ 41.56217205, 110.13144155,  73.52860638,  -0.55006384])

Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [33]:
from sklearn.linear_model import LinearRegression
 
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
 
print(model.coef_)

[[ 41.56217205 110.13144155  73.52860638  -0.55006384]]


Если вы все сделали верно, они должны практически совпасть!

## Стохастический градиентный спуск (6 баллов)

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы не нормализовали еще!). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [34]:
#ваш код
y = df['sales'].values
X = df.drop('sales', axis=1)

X = (X - X.mean())/X.std()
X.head()

,TV,radio,newspaper
0,0.967425,0.979066,1.774493
1,-1.194379,1.080097,0.667903
2,-1.512360,1.524637,1.779084
3,0.051919,1.214806,1.283185
4,0.393196,-0.839507,1.278593


**Добавим единичный столбец**

In [35]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
X[0]

array([1.        , 0.9674246 , 0.97906559, 1.77449253])

**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [36]:
#ваш код
def mse_error(y, y_pred):
    error = 0
    for i in range(y.shape[0]):
        error += (y_pred - y[i])**2
    mse = error/y.shape[0]
    return mse

**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [37]:
#ваш код
y_pred = y.mean()
print('Mean of y:', y_pred)
print('MSE:', mse_error(y, y_pred))

Mean of y: 14.0225
MSE: 27.085743749999985


**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [38]:
#ваш код
def lin_pred(X, w):
    return np.dot(X, w)

**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла) 
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [39]:
#ваш код
L = X.shape[1]
def stoch_grad_step(X, y, w, train_ind, eta):
    der = (2*X[train_ind]*(X[train_ind]@ w - y[train_ind]))/L
    return w - eta*der

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу 

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте основной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [40]:
# ваш код
def stochastic_gradient_descent(X, y, w, eta, n_iter, min_weight_dist):
    weight_dist = np.inf
    costs = []
    iter = 0
    w = w_init
    for i in range(n_iter):
        train_ind = np.random.randint(0, X.shape[0])
        w_prev = w
        w = stoch_grad_step(X, y, w, train_ind, eta)
        weight_dist = np.linalg.norm(w - w_prev)
        costs.append(((X[train_ind]@w - y[train_ind])**2)/X.shape[0])  
        iter += 1
        if weight_dist < min_weight_dist:
            break  
    return w, costs[-1]

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

In [41]:
w_init = np.zeros(X.shape[1])
stochastic_gradient_descent(X, y, w, 0.01, 100000, 0.00001)

(array([13.93398884,  3.86369196,  3.13925238,  0.0147028 ]),
 8.816821250132427e-10)

=======================

**Постройте график зависимости ошибки от номера итерации**

In [42]:
# ваш код
def stochastic_gradient_descent2(X, y, w, eta, n_iter, min_weight_dist):
    weight_dist = np.inf
    costs = []
    init = 0
    iter = []
    w = w_init
    for i in range(n_iter):
        train_ind = np.random.randint(0, X.shape[0])
        w_prev = w
        w = stoch_grad_step(X, y, w, train_ind, eta)
        weight_dist = np.linalg.norm(w - w_prev)
        costs.append(((X[train_ind]@w - y[train_ind])**2)/X.shape[0])  
        init += 1
        iter.append(init)
        if weight_dist < min_weight_dist:
            break
           
    return costs, iter


import plotly
import plotly.express as px

fig = px.line(
    x = stochastic_gradient_descent2(X, y, w, 0.01, 100000, 0.00001)[1][0:1000],
    y = stochastic_gradient_descent2(X, y, w, 0.01, 100000, 0.00001)[0][0:1000],
    height = 500,
    width = 1000,
)

fig.update_xaxes(title_text='Iterations')
fig.update_yaxes(title_text='MSE')
fig.show()

**Выведите вектор весов, к которому сошелся метод.**

In [43]:
# ваш код
display(stochastic_gradient_descent(X, y, w, 0.01, 100000, 0.00001)[0])

array([13.96510659,  3.96442547,  2.70201188, -0.01580438])

**Выведите среднеквадратичную ошибку на последней итерации.**

In [44]:
# ваш код
display(stochastic_gradient_descent(X, y, w, 0.01, 100000, 0.00001)[1])

3.5623416058084122e-09